In [1]:
%pip install pandas psycopg2 sqlalchemy ipykernel
%pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 435.7 kB/s eta 0:00:03
   -- ------------------------------------- 0.1/1.2 MB 544.7 kB/s eta 0:00:03
   --------------- ------------------------ 0.4/1.2 MB 2.5 MB/s eta 0:00:01
   ---------------------------------------  1.2/1.2 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 4.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
import psycopg2
from psycopg2 import extensions as _ext
import pandas as pd
from sqlalchemy import create_engine
import select

# Database connection details
username = 'postgres'
password = 'root'
host = 'localhost'
port = '5433'
database = 'final_project'

# Creating a connection string for SQLAlchemy
connection_string = f'postgresql://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(connection_string)

# Function to create raw CSV files from the database tables
def creating_raw():
    query = 'SELECT * FROM public."Users"'
    df1 = pd.read_sql(query, engine)
    df1.to_csv('RAW_Users.csv', index=False)

    query2 = 'SELECT * FROM public."Courses"'
    df2 = pd.read_sql(query2, engine)
    df2.to_csv('RAW_Courses.csv', index=False)

    query3 = 'SELECT * FROM public."Feedbacks"'
    df3 = pd.read_sql(query3, engine)
    df3.to_csv('RAW_Feedbacks.csv', index=False)

    query4 = 'SELECT * FROM public."TrainingPerformances"'
    df4 = pd.read_sql(query4, engine)
    df4.to_csv('RAW_TrainingPerformances.csv', index=False)

creating_raw()

# Function to listen for notifications from PostgreSQL
def listen_to_notifications():
    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(
        dbname=database,
        user=username,
        password=password,
        host=host,
        port=port
    )
    print(conn)
    # Set the isolation level for the connection
    conn.set_isolation_level(_ext.ISOLATION_LEVEL_AUTOCOMMIT)

    # Create a cursor object using the connection
    cursor = conn.cursor()

    # Execute the LISTEN command to listen for notifications on 'table_update'
    cursor.execute("LISTEN table_update;")
    print("Listening for notifications on table_update...")

    # Infinite loop to continuously listen for notifications
    while True:
        # Wait for notifications with a timeout
        if select.select([conn], [], [], 5) == ([], [], []):
            print("Waiting for notifications...")
        else:
            # Process notifications if any are available
            conn.poll()
            while conn.notifies:
                notify = conn.notifies.pop(0)
                print(f"Notification received: {notify.payload}")
                # Call the function to create raw CSV files
                creating_raw()

if __name__ == "__main__":
    listen_to_notifications()


<connection object at 0x00000217FD86E020; dsn: 'user=postgres password=xxx dbname=final_project host=localhost port=5433', closed: 0>
Listening for notifications on table_update...
Waiting for notifications...
Waiting for notifications...
Waiting for notifications...
Waiting for notifications...
Waiting for notifications...
Waiting for notifications...
Notification received: Feedbacks changed
Waiting for notifications...
Waiting for notifications...
Waiting for notifications...
Waiting for notifications...
Waiting for notifications...
Waiting for notifications...


KeyboardInterrupt: 